# This notebook contains the deep learning models fo the car price prediction task.

In [6]:
# Import required packages 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from keras import layers, Sequential
from keras.layers import Dense, Input
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [7]:
# Importing the preprocessed data
data = pd.read_csv("preprocessed_data.csv")
data.head()

,car_name,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [8]:
# Encoding the car_name column before splitting to avoid not seen label error 

encoder = LabelEncoder()

data["car_name"]=data["car_name"].apply(str.lower)

data["car_name"] = encoder.fit_transform(data["car_name"])

np.save('./app/car_name.npy', encoder.classes_)

In [9]:
# Split the data into features and labels
features = data.drop("selling_price", axis=1)
labels = data["selling_price"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(12328, 12328, 3083, 3083)

In [10]:
# convert non-numerical columns to numerical columns

cat_columns = ["seller_type", "fuel_type", "transmission_type"]

for label in cat_columns:
    encoder = LabelEncoder()
    
    # X_train[label]=X_train[label].apply(str.lower)
    # X_val[label]=X_val[label].apply(str.lower)
        
    X_train[label] = encoder.fit_transform(X_train[label])
    X_val[label] = encoder.transform(X_val[label])
    np.save(f'./app/{label}.npy', encoder.classes_)

In [11]:
# convert dataframe to array
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()

In [12]:
# Scaling the dataset

scaler = StandardScaler()

# Reshaping the labels to avoid shape error when fitting scaler
y_train = y_train.to_numpy().reshape(-1, 1)
y_val = y_val.to_numpy().reshape(-1, 1)

scaler.fit(y_train)

# Scaling
# y_train = scaler.transform(y_train)
# y_val = scaler.transform(y_val)

StandardScaler()

In [13]:
X_train[0].shape

(10,)

In [14]:
model = Sequential()
model.add(Input(shape=(X_train[0].shape)))
model.add(Dense(32, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1))
# compile model
model.compile(loss='mae', optimizer=optimizers.Adam(learning_rate=2e-3))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train, y_train, epochs=1000, validation_data=(X_val, y_val),
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=20), tf.keras.callbacks.ModelCheckpoint("./app/checkpoints/model_checkpoint", save_best_only=True, save_weights_only=True)])

Epoch 1/1000
386/386 [==============================] - 2s 3ms/step - loss: 584601.9375 - val_loss: 536436.8125
Epoch 2/1000
386/386 [==============================] - 1s 2ms/step - loss: 470709.0312 - val_loss: 415930.5938
Epoch 3/1000
386/386 [==============================] - 1s 2ms/step - loss: 309645.9688 - val_loss: 306678.3125
Epoch 4/1000
386/386 [==============================] - 1s 3ms/step - loss: 288983.1875 - val_loss: 303500.2812
Epoch 5/1000
386/386 [==============================] - 1s 2ms/step - loss: 288006.5938 - val_loss: 302269.1875
Epoch 6/1000
386/386 [==============================] - 1s 2ms/step - loss: 286530.1250 - val_loss: 301643.6562
Epoch 7/1000
386/386 [==============================] - 1s 3ms/step - loss: 285333.6250 - val_loss: 300564.3750
Epoch 8/1000
386/386 [==============================] - 1s 2ms/step - loss: 284126.5938 - val_loss: 301769.4688
Epoch 9/1000
386/386 [==============================] - 1s 3ms/step - loss: 283186.0000 - val_loss: 3007

In [16]:
model.load_weights("./app/checkpoints/model_checkpoint")
preds = model.predict(X_val)
print(preds[:5])
scaler.inverse_transform(preds[0].reshape(1, -1))

97/97 [==============================] - 0s 1ms/step
[[ 233223.47]
 [ 558349.5 ]
 [ 476534.5 ]
 [1692466.6 ]
 [ 122750.7 ]]


array([[2.1003592e+11]], dtype=float32)

In [17]:
print(y_val[:5])
scaler.inverse_transform(y_val[0].reshape(1, -1))

[[ 190000]
 [ 600000]
 [ 665000]
 [1570000]
 [ 160000]]


array([[1.71109965e+11]])

In [18]:
!cd app

In [22]:
from app.model import build_model

loaded_model = build_model()
loaded_model.load_weights('./app/checkpoints/model_checkpoint')

loaded_model.predict(X_val)[:5]

97/97 [==============================] - 0s 2ms/step


array([[ 233223.47],
       [ 558349.5 ],
       [ 476534.5 ],
       [1692466.6 ],
       [ 122750.7 ]], dtype=float32)

In [28]:
loaded_model.predict(tf.expand_dims(X_val[0], axis=0))

1/1 [==============================] - 0s 81ms/step


array([[233223.47]], dtype=float32)

In [19]:
# This code is taken from geeks for geeks page (https://www.geeksforgeeks.org/convert-the-number-from-international-system-to-indian-system/)
def convert(input):

	# Find the length of the
	# input string
	Len = len(input)

	# Removing all the separators(, )
	# from the input string
	i = 0
	while(i < Len):
		if(input[i] == ","):
			input = input[:i] + input[i + 1:]
			Len -= 1
			i -= 1
		elif(input[i] == " "):
			input=input[:i] + input[i + 1:]
			Len -= 1
			i -= 1
		else:
			i += 1
	# Reverse the input string
	input=input[::-1]

	# Declaring the output string
	output = ""

	# Process the input string
	for i in range(Len):

		# Add a separator(, ) after the
		# third number
		if(i == 2):
			output += input[i]
			output += ","
		
		# Then add a separator(, ) after
		# every second number
		elif(i > 2 and i % 2 == 0 and
			i + 1 < Len):
			output += input[i]
			output += ","
		else:
			output += input[i]
	
	# Reverse the output string
	output=output[::-1]

	# Return the output string back
	# to the main function
	return output

In [33]:
prediction = tf.round(model.predict(tf.expand_dims(X_val[0], axis=0)))
print(convert(str(prediction.numpy()[0][0])))

1/1 [==============================] - 0s 36ms/step
2,33,22,3.0


In [38]:
import sklearn
import numpy as np
import tensorflow
import keras
import gradio


gradio.__version__

'3.15.0'

In [2]:
!pip freeze > requirements.txt